In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv('train_bigmarket.csv')
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [2]:
train=train[['Item_Visibility','Item_MRP','Outlet_Type','Item_Outlet_Sales']]
train.columns=['vis','mrp','otype','sale']
train.head()

,vis,mrp,otype,sale
0,0.016047,249.8092,Supermarket Type1,3735.1380
1,0.019278,48.2692,Supermarket Type2,443.4228
2,0.016760,141.6180,Supermarket Type1,2097.2700
3,0.000000,182.0950,Grocery Store,732.3800
4,0.000000,53.8614,Supermarket Type1,994.7052


In [3]:
train.loc[train.vis<.2,'vis']=0
train.loc[train.vis>=.2,'vis']=1

In [4]:
train['otype'].replace({'Supermarket Type1':2,'Supermarket Type2':1,'Supermarket Type3':3,'Grocery Store':0},inplace=True)

In [5]:
train.head()

,vis,mrp,otype,sale
0,0.0,249.8092,2,3735.1380
1,0.0,48.2692,1,443.4228
2,0.0,141.6180,2,2097.2700
3,0.0,182.0950,0,732.3800
4,0.0,53.8614,2,994.7052


In [6]:
x=pd.get_dummies(columns=['vis','otype'],data=train.drop('sale',axis=1))
x.drop(['vis_1.0'],axis=1,inplace=True)
y=train.sale
x.head()

,mrp,vis_0.0,otype_0,otype_1,otype_2,otype_3
0,249.8092,1,0,0,1,0
1,48.2692,1,0,1,0,0
2,141.6180,1,0,0,1,0
3,182.0950,1,1,0,0,0
4,53.8614,1,0,0,1,0


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kf=KFold(n_splits=10,random_state=1,shuffle=True)

In [8]:
score=cross_val_score(LinearRegression(),x,y,cv=kf).mean()
score

0.5618397981298549

In [9]:
score=cross_val_score(DecisionTreeRegressor(max_leaf_nodes=18),x,y,cv=kf).mean()
score

0.5924077254294458

In [10]:
score=cross_val_score(RandomForestRegressor(max_leaf_nodes=18),x,y,cv=kf).mean()
score

0.5983321763051197

In [11]:
score=cross_val_score(RandomForestRegressor(max_depth=5),x,y,cv=kf).mean()
score

0.5997308959096325

In [12]:
score=cross_val_score(RandomForestRegressor(max_leaf_nodes=22,max_depth=5),x,y,cv=kf).mean()
score

0.5995923089617546

In [13]:
rg=RandomForestRegressor(max_leaf_nodes=22,max_depth=5)
rg.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=22,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [14]:
test=pd.read_csv('test_bigmarket.csv')
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [15]:
test=test[['Item_Visibility','Item_MRP','Outlet_Type']]
test.columns=['vis','mrp','otype']

In [16]:
test.loc[test.vis<.2,'vis']=0
test.loc[test.vis>=.2,'vis']=1

In [17]:
test['otype'].replace({'Supermarket Type1':2,'Supermarket Type2':1,'Supermarket Type3':3,'Grocery Store':0},inplace=True)

In [18]:
test=pd.get_dummies(columns=['vis','otype'],data=test)
test.drop(['vis_1.0'],axis=1,inplace=True)

In [19]:
test['sale']=rg.predict(test)
test.head()

,mrp,vis_0.0,otype_0,otype_1,otype_2,otype_3,sale
0,107.8622,1,0,0,1,0,1557.246698
1,87.3198,1,0,0,1,0,1458.481204
2,241.7538,1,1,0,0,0,563.162716
3,155.0340,1,0,0,1,0,2515.280798
4,234.2300,1,0,0,0,1,6214.064536


In [20]:
test['sale']=rg.predict(test.drop('sale',axis=1))
test.head()

,mrp,vis_0.0,otype_0,otype_1,otype_2,otype_3,sale
0,107.8622,1,0,0,1,0,1557.246698
1,87.3198,1,0,0,1,0,1458.481204
2,241.7538,1,1,0,0,0,563.162716
3,155.0340,1,0,0,1,0,2515.280798
4,234.2300,1,0,0,0,1,6214.064536


In [21]:
submit=pd.read_csv('sample_submission_bigmarket.csv')
submit.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000


In [22]:
submit.Item_Outlet_Sales=test.sale
submit.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1557.246698
1,FDW14,OUT017,1458.481204
2,NCN55,OUT010,563.162716
3,FDQ58,OUT017,2515.280798
4,FDY38,OUT027,6214.064536


In [23]:
submit.to_csv('randforest_2.csv',index=False)